In [1]:
# Install the packages
%pip install --user --quiet google-cloud-aiplatform \
                         google-cloud-storage \
                         google-cloud-pipeline-components \
                         kfp


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import sys
from IPython.display import Markdown, display

PROJECT_ID="ai-hangsik"
LOCATION="us-central1"
BUCKET_URI = f"gs://mlops-poc-0303"
PIPELINE_ROOT = f"{BUCKET_URI}/pipeline/"

# For only colab user, no need this process for Colab Enterprise in Vertex AI.
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id=PROJECT_ID)

# set project.
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [3]:
import json
from typing import NamedTuple

from google.cloud import aiplatform
from kfp import compiler, dsl
from kfp.dsl import component
from google.cloud import aiplatform, bigquery

aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [5]:
shell_output = ! gcloud projects describe  $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")

SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

SERVICE_ACCOUNT: 721521243942-compute@developer.gserviceaccount.com


In [55]:
def build_pipeline ( pipeline_name:str,
                     pipeline_desc:str,
                     pipeline_root:str, 
                     data_comp_file:str,
                     training_comp_file:str,
                     serving_comp_file:str):

    @dsl.pipeline(
        name = pipeline_name,
        description = pipeline_desc,
        pipeline_root = pipeline_root,)

    def module_pipeline(text: str, ):

        from kfp import components

        data_component = components.load_component_from_file(data_comp_file)
        training_component = components.load_component_from_file(training_comp_file)
        serving_component = components.load_component_from_file(serving_comp_file)

        data_output = data_component(text=text)
        training_output = training_component(text=data_output.output)
        serving_output = serving_component(text=training_output.output)

    compiler.Compiler().compile(pipeline_func=module_pipeline, package_path="module_pipeline.yaml")


In [56]:
build_pipeline( pipeline_name = "module_pipeline",
                pipeline_desc = "desc for module_pipeline",
                pipeline_root = PIPELINE_ROOT,

                data_comp_file = 'data_prep.yaml',
                training_comp_file = 'training.yaml',
                serving_comp_file = 'serving.yaml' 
              )

In [57]:
text = "Hi this is moudule pipeline test"

job = aiplatform.PipelineJob(

    display_name="module_pipeline",
    template_path="module_pipeline.yaml",
    parameter_values = {"text": text,
                       },
    pipeline_root=PIPELINE_ROOT,
    enable_caching = False
)

job.run(service_account = SERVICE_ACCOUNT)

Creating PipelineJob
PipelineJob created. Resource name: projects/721521243942/locations/us-central1/pipelineJobs/module-pipeline-20250305082347
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/721521243942/locations/us-central1/pipelineJobs/module-pipeline-20250305082347')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/module-pipeline-20250305082347?project=721521243942
PipelineJob projects/721521243942/locations/us-central1/pipelineJobs/module-pipeline-20250305082347 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/721521243942/locations/us-central1/pipelineJobs/module-pipeline-20250305082347 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/721521243942/locations/us-central1/pipelineJobs/module-pipeline-20250305082347 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/721521243942/locations/us-central1/pipelineJobs/mo